In [1]:
using Random, Distributions

In [2]:
struct Risk 
    Id::Int
    Name::String
    Exposure::Float64
    MeanPoD::Float64
    Rating::Int
    AccumulationId::Int
    Region::Int
    Sector::Int
end

In [3]:
struct Status
    Id::Int
    Probability::Float64
    PoDModifiers::Vector{Float64}
end 

In [4]:
struct Region
    Id::Int
    Name::String
    Status::Status
end

In [5]:
struct Sector
    Id::Int
    Name::String
    Status::Status
end

In [6]:
struct GlobalShock
    PoDModifies::Vector{Float64}
end

In [7]:
struct Scenario
    Id::Int
    GlobalShock::Bool
    Regions::Vector{Region}
    Sectors::Vector{Sector}
end

In [8]:
function select_random_entry(entries::Vector)
    r = rand() * length(entries)
    index = findfirst(i -> i > r, 1:length(entries))
    return index
end

select_random_entry (generic function with 1 method)

In [9]:
function generate_regions(regions_names::Vector, statuses::Vector)
    regions = []
    for (index, region_name) in enumerate(regions_names)
        r = rand()
        selected_status = statuses[findfirst(s -> s.Probability > r, statuses)]
        push!(regions, Region(index, region_name, selected_status))
    end
    return regions
end

generate_regions (generic function with 1 method)

In [10]:
function generate_sectors(sectors_names::Vector, statuses::Vector)
    sectors = []
    for (index, sector_name) in enumerate(sectors_names)
        r = rand()
        selected_status = statuses[findfirst(s -> s.Probability > r, statuses)]
        push!(sectors, Sector(index, sector_name, selected_status))
    end
    return sectors
end

generate_sectors (generic function with 1 method)

In [11]:
function generate_scenarios(number_of_scenarios, regions_names, secors_names)
    scenarios = [Scenario(i, false, generate_regions(regions_names, statuses), generate_sectors(sectors_names, statuses)) for i in 1:number_of_scenarios]
end

generate_scenarios (generic function with 1 method)

In [12]:
function generate_risks(number_of_risks, region_names, sector_names, ratings, exposure_distribution)
    return [Risk(i, randstring(), rand(exposure_distribution) , 0.005 + (rand() / 10), select_random_entry(ratings),
        1, select_random_entry(regions_names), select_random_entry(sectors_names)) for i in 1:number_of_risks]
end

generate_risks (generic function with 1 method)

In [13]:
regions_names = ["Europe", "North America", "South America", "Africa", "Asia", "Oceania"];

sectors_names = ["Agriculture", "Raw Materials Production", "Energy production", "Heavy Industry", "Clothing Industry", "Tech Industry", "Logistics", "Other Services"];

ratings = ["A", "B+", "B", "C"];

statuses = [Status(1, 0.57, [0.3, 0.4, 0.6, 0.7]),
    Status(2, 0.70, [0.3, 0.4, 0.6, 0.7]),
    Status(3, 0.77, [0.3, 0.4, 0.6, 0.7]),
    Status(4, 0.86, [0.3, 0.4, 0.6, 0.7]),
    Status(5, 0.94, [0.3, 0.4, 0.6, 0.7]),
    Status(6, 1, [0.3, 0.4, 0.6, 0.7])];

In [22]:
#generate risks
risks = generate_risks(1000, regions_names, sectors_names, ratings, Normal(1000000, 200000))

#generate scenarios
scenarios = generate_scenarios(100000, regions_names, sectors_names);

In [23]:
function modifier(id::Int, rating::Int, selection::Vector, statuses::Vector)
    selected = selection[findfirst(s -> s.Id == id, selection)]
    return (statuses[selected.Status.Id]).PoDModifiers[rating]
end

function bankrupt(risk, scenario, statuses)
    pod = risk.MeanPoD * modifier(risk.Region, risk.Rating, scenario.Regions, statuses) * modifier(risk.Sector, risk.Rating, scenario.Sectors, statuses)
    return rand() < pod;
end

function loss(risk)
    return rand(Beta(2, 5)) * risk.Exposure
end

loss (generic function with 1 method)

In [24]:
function calculate_loss_per_risk_per_scenario(risks, scenarios)
    return [bankrupt(risk, scenario, statuses) ? loss(risk) : 0 for risk in risks for scenario in scenarios]
end

calculate_loss_per_risk_per_scenario (generic function with 1 method)

In [25]:
res = calculate_loss_per_risk_per_scenario(risks, scenarios)

total_loss = sum(res)

4.1620791438143665e11